# MatchaGet Helper
### 2026 Made with ❤️, Gilang Wahyu Prasetyo
Mo ambil riwayat GC seluruh SBR? pake Tools ini ges

# 1. Improt

In [4]:
import os
import re
import time
import random
import requests
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
pd.set_option("display.max_columns", None)



# 2. Buka Chrome

In [2]:
url = "https://matchapro.web.bps.go.id/dirgc"

chrome_cust = "isi di sini path nya"

user_agent = (
    "Mozilla/5.0 (Linux; Android 15; SM-G991B Build/TP1A.220624.014; wv) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 "
    "Chrome/132.0.6834.88 Mobile Safari/537.36"
)

options = Options()
options.add_argument("--start-maximized")
options.add_argument(f"--user-agent={user_agent}")

try:
    service = Service(executable_path=chrome_cust)
    driver = webdriver.Chrome(service=service, options=options)
except Exception:
    service = Service()
    driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

# 3. Login SSO
masukin OTP dulu

In [ ]:
username_sso = "sso"
password_sso = "pass"

wait = WebDriverWait(driver, 20)

# Klik tombol SSO
wait.until(EC.element_to_be_clickable((By.ID, "login-sso"))).click()

# Isi kredensial
wait.until(EC.presence_of_element_located((By.ID, "username"))).send_keys(username_sso)
driver.find_element(By.ID, "password").send_keys(password_sso)

# Submit login
driver.find_element(By.ID, "kc-login").click()

# 4. Ambil Cookies n Token
pastikan kamu di halaman dirgc

In [ ]:
# Tunggu login selesai
time.sleep(5)

# Ambil cookies dari Selenium
cookies_selenium = driver.get_cookies()

# Ambil CSRF token
csrf_token = driver.execute_script("""
    return document.querySelector('meta[name="csrf-token"]').content;
""")

print(f"CSRF Token: {csrf_token}")

# Convert cookies ke requests session
session = requests.Session()
for cookie in cookies_selenium:
    session.cookies.set(cookie['name'], cookie['value'])


# 5. Crawl

## 5.1 tes seribu dulu (kali aja gagal ye kan)
coba jalanin ini dulu. kalo berhasil, baru jalanin 5.2. ga wajib sih, jaga2 aja.

In [9]:
# Jangan pakai requests, pakai driver.execute_script
data_all = driver.execute_script("""
    return fetch('https://matchapro.web.bps.go.id/direktori-usaha/data-gc-card', {
        method: 'POST',
        headers: {'X-Requested-With': 'XMLHttpRequest'},
        body: new URLSearchParams({
            '_token': document.querySelector('meta[name="csrf-token"]').content,
            'start': '0',
            'length': '1000',
            'status_filter': 'semua'
        })
    }).then(r => r.json());
""")

print(data_all)


{'data': [{'alamat_usaha': 'JL. jend. a. yani, mabuun', 'alamat_usaha_gc': None, 'allow_cancel': False, 'allow_edit': False, 'allow_flagging': False, 'gc_username': 'dessyp', 'gcid': 'eyJpdiI6ImdjclgzaW9mN3RoU0kwQ3BNbC9HOXc9PSIsInZhbHVlIjoidUQ3blkyZGhiSEVRSUZCaXprMndodz09IiwibWFjIjoiMzU5ODA2NjEwYmE1ZGI4ODNiYzBhMTYwMGQ1ZDgyYTAzOTBjMzY1YmI5ZGYzZDliZjc5M2VlNmQ0YTllYmQ3MSIsInRhZyI6IiJ9', 'gcs_result': '1', 'history_ref_profiling_id': '2025-06-12', 'idsbr': '13635', 'kddesa': '003', 'kdkab': '09', 'kdkec': '070', 'kdprov': '63', 'kegiatan_usaha': '[Kegiatan Usaha: penyedia akomodasi perhotelan, Kategori: I, KBLI: 55110]; [Kegiatan Usaha: restoran...', 'kode_wilayah': '6309070003', 'latitude': '-2.17673', 'latitude_gc': '-2.17673', 'latlong_status': 'valid', 'latlong_status_gc': 'valid', 'longitude': '115.42373', 'longitude_gc': '115.42373', 'nama_usaha': 'CHANDRA BATUAH ALAM LESTARI, PT', 'nama_usaha_gc': None, 'nmdesa': 'MABUUN', 'nmkab': 'TABALONG', 'nmkec': 'MURUNG PUDAK', 'nmprov': 'KAL

## 5.2 gas semua data

In [13]:
all_data = []
start = 0
length = 1000

start_time = time.time()
last_time = start_time

while True:
    fetch_start = time.time()
    print(f"Fetching start={start}...")
    
    try:
        result = driver.execute_script(f"""
            return fetch('https://matchapro.web.bps.go.id/direktori-usaha/data-gc-card', {{
                method: 'POST',
                headers: {{'X-Requested-With': 'XMLHttpRequest'}},
                body: new URLSearchParams({{
                    '_token': document.querySelector('meta[name="csrf-token"]').content,
                    'start': '{start}',
                    'length': '{length}',
                    'provinsi': '127',
                    'kabupaten': '2512',
                    'status_filter': 'semua',
                    'nama_usaha': '',
                    'alamat_usaha': '',
                    'kecamatan': '',
                    'desa': '',
                    'rtotal': '0',
                    'sumber_data': '',
                    'skala_usaha': '',
                    'idsbr': '',
                    'history_profiling': '',
                    'f_latlong': '',
                    'f_gc': ''
                }})
            }}).then(r => r.json());
        """)
        
        fetch_duration = time.time() - fetch_start
        
        data = result.get("data", [])
        if not data:
            print("Server returned no more data")
            break
        
        all_data.extend(data)
        elapsed = time.time() - start_time
        time_per_row = elapsed / len(all_data)
        print(f"Progress: {len(all_data)} rows | Fetch: {fetch_duration:.1f}s | Elapsed: {elapsed:.1f}s | {time_per_row*1000:.1f}ms/row")
        
    except Exception as e:
        print(f"Error: {e}")
        break
    
    start += length
    time.sleep(random.uniform(2, 5))

# Save
total_time = time.time() - start_time
df = pd.DataFrame(all_data)
time_per_row = total_time / len(df) if len(df) > 0 else 0
print(f"\nTotal: {len(df)} rows | Time: {total_time:.1f}s ({total_time/60:.1f} minutes) | Avg: {time_per_row*1000:.1f}ms/row")
df.to_excel("matcha_crawl.xlsx", index=False)
driver.quit()


Fetching start=0...
Progress: 1000 rows | Fetch: 1.1s | Elapsed: 1.1s | 1.1ms/row
Fetching start=1000...
Progress: 2000 rows | Fetch: 1.1s | Elapsed: 4.3s | 2.1ms/row
Fetching start=2000...
Progress: 3000 rows | Fetch: 1.1s | Elapsed: 9.1s | 3.0ms/row
Fetching start=3000...
Progress: 4000 rows | Fetch: 1.1s | Elapsed: 13.4s | 3.3ms/row
Fetching start=4000...
Progress: 5000 rows | Fetch: 1.1s | Elapsed: 18.2s | 3.6ms/row
Fetching start=5000...
Progress: 6000 rows | Fetch: 1.2s | Elapsed: 21.5s | 3.6ms/row
Fetching start=6000...
Progress: 7000 rows | Fetch: 1.2s | Elapsed: 26.2s | 3.7ms/row
Fetching start=7000...
Progress: 8000 rows | Fetch: 1.0s | Elapsed: 30.3s | 3.8ms/row
Fetching start=8000...
Progress: 9000 rows | Fetch: 1.7s | Elapsed: 34.0s | 3.8ms/row
Fetching start=9000...
Progress: 10000 rows | Fetch: 1.0s | Elapsed: 38.3s | 3.8ms/row
Fetching start=10000...
Progress: 11000 rows | Fetch: 1.0s | Elapsed: 43.5s | 4.0ms/row
Fetching start=11000...
Progress: 12000 rows | Fetch: 1.1

# 6. Save ke lokal

In [ ]:
df = pd.DataFrame(all_data)
print(f"\nTotal: {len(df)} rows | Time: {total_time:.1f}s ({total_time/60:.1f} minutes) | Avg: {time_per_row*1000:.1f}ms/row")
df.to_excel(r"C:\Users\acer\Downloads\matchaget.xlsx", index=False)
driver.quit()